In [ ]:
!pip install transformers
!pip install rouge-score
!pip install openai
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e8ffe0383d1613bccdd3f8425a4da9f89107651d54df07977e94c16e11160dfe
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import requests
import xml.etree.ElementTree as ET
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import nltk
from nltk.translate import meteor_score
from nltk.tokenize import word_tokenize
nltk.download('punkt')  # Download the 'punkt' resource
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Path to the XML file
xml_file_path = "/content/drive/MyDrive/Elsevier /LG LLM/topics2016.xml"

# Parse the XML file
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Extract the clinical notes and reference summaries
clinical_notes = []
reference_summaries = []
for topic in root.findall("topic"):
    description = topic.find("description").text
    clinical_notes.append(description)
    summary = topic.find("summary").text
    reference_summaries.append(summary)

tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-bigpatent")
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-bigpatent")

In [ ]:
# Initialize BLEU and ROUGE scorers
bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []
meteor_scores = []

rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'])

# Summarize clinical notes and evaluate
for i, note in enumerate(clinical_notes):
    inputs = tokenizer.encode(note, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, num_beams = 4, max_length=100, early_stopping=True)
    summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)

    # Evaluate BLEU score
    bleu_score = corpus_bleu([[reference_summaries[i].split()]], [summary.split()])
    bleu_scores.append(bleu_score)

    # Evaluate ROUGE scores
    reference = reference_summaries[i]
    hypothesis = summary
    rouge_scores = rouge_scorer.score(reference, hypothesis)

    rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
    rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
    rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

    # Tokenize hypothesis and reference summaries
    hypothesis_tokens = word_tokenize(hypothesis)
    reference_tokens = word_tokenize(reference)

    # Evaluate METEOR score
    meteor_score_value = meteor_score.meteor_score([reference_tokens], hypothesis_tokens)
    meteor_scores.append(meteor_score_value)

    print("Clinical Note:")
    print(note)
    print("\nReference Summary:")
    print(reference_summaries[i])
    print("\nGenerated Summary:")
    print(summary)
    print("\nBLEU Score:", bleu_score)
    print("ROUGE Scores:", rouge_scores)
    print("METEOR Score:", meteor_score_value)
    print("--------------------------------------------------")

Attention type 'block_sparse' is not possible if sequence_length: 62 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram

Clinical Note:
78 M transferred to nursing home for rehab after CABG. Reportedly readmitted with a small NQWMI. Yesterday, he was noted to have a melanotic stool and then today he had approximately 9 loose BM w/ some melena and some frank blood just prior to transfer, unclear quantity.

Reference Summary:
A 78 year old male presents with frequent stools and melena.

Generated Summary:
A patient has been admitted to a hospital.

BLEU Score: 7.445183326929416e-232
ROUGE Scores: {'rouge1': Score(precision=0.125, recall=0.09090909090909091, fmeasure=0.10526315789473685), 'rouge2': Score(precision=0.0, recall=0.0, fmeasure=0.0), 'rougeL': Score(precision=0.125, recall=0.09090909090909091, fmeasure=0.10526315789473685)}
METEOR Score: 0.08547008547008547
--------------------------------------------------
Clinical Note:
An elderly female with past medical history of hypertension, severe aortic stenosis, hyperlipidemia, and right hip arthroplasty. Presents after feeling a snap of her right leg 

In [ ]:
# Calculate and print average evaluation scores
avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
avg_rouge1_score = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2_score = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL_score = sum(rougeL_scores) / len(rougeL_scores)
avg_meteor_score = sum(meteor_scores) / len(meteor_scores)

print("Average BLEU Score:", avg_bleu_score)
print("Average ROUGE-1 Score:", avg_rouge1_score)
print("Average ROUGE-2 Score:", avg_rouge2_score)
print("Average ROUGE-L Score:", avg_rougeL_score)
print("Average METEOR Score:", avg_meteor_score)

Average BLEU Score: 0.016776738903392298
Average ROUGE-1 Score: 0.1841119191861065
Average ROUGE-2 Score: 0.05912326352728004
Average ROUGE-L Score: 0.1518346348202554
Average METEOR Score: 0.13126347246576053


In [ ]:
##finetuned code